# MSDS Marketing Text Analytics, Unit 2, Assignment 2: Build a topic model

## ⚡️ Make a Copy

Save a copy of this notebook in your Google Drive before continuing. Be sure to edit your own copy, not the original notebook.

In this assignment, you will implement a topic model preprocessor which can then be applied to the task of topic-modeling Amazon text reviews. Please review the course lectures and documentation up to this point before continuing. Be sure also to be familiar with the [documentation for TMToolkit](https://tmtoolkit.readthedocs.io/en/latest/topic_modeling.html)

Be sure to make a copy into your own Drive account before editing this notebook.

You will implement a preprocessing function to prepare your corpus for topic modeling. It is recommended that you use a small test corpus (an example is provided below) for development, rather than starting with the full review set.

---

## ⚠️ Important Note

This notebook has been updated to reflect **significant changes to the tmtoolkit API**. You will find some differences from other course materials, including the lectures. There is no longer a TMPreproc object in tmtoolkit. Rather, preprocessing functions have been moved into the tmtoolkit.corpus module. All of the preprocessing functions you will need for this assignment are imported for you below.

---

## Dependency installs

**Important:** You will likely see a message to restart the runtime after the installations are complete, and should do so.

### Remove some libraries in Colab that cause conflicts

In [1]:
!pip uninstall -y numba
!pip uninstall -y tensorflow

Found existing installation: numba 0.58.1
Uninstalling numba-0.58.1:
  Successfully uninstalled numba-0.58.1
Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0


### Install LDA and tmtoolkit

In [2]:
!pip install lda
!pip install "tmtoolkit[recommended]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.8/349.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 7.6 MB/s eta 0:00:00
  Created wheel for globre: filename=globre-0.1.5-py3-none-any.whl size=19546 sha256=940d01f87d0dd21691dea9e2efb3d60327ae6ce0e1b0810153f0e5b5dc73189f
  Stored in directory: /root/.cache/pip/wheels/e6/c5/26/0d27e07d567b3b4e63a618efff19a2cdbab636ac1295d20206
Successfully built globre


## Imports


In [3]:
# This is just here for documentation purposes: tmtoolkit no longer has
# a TMPreproc. Don't use it for this assignment! Instead use the functions
# that are imported below.

# Do not do this:
#from tmtoolkit.preprocess import TMPreproc

In [17]:
from tmtoolkit.corpus import Corpus, lemmatize, to_lowercase, remove_chars, filter_clean_tokens
from tmtoolkit.corpus import corpus_num_tokens, corpus_tokens_flattened
from tmtoolkit.corpus import dtm
from tmtoolkit.corpus import vocabulary
from tmtoolkit.topicmod.model_io import print_ldamodel_topic_words
from tmtoolkit.topicmod.tm_lda import compute_models_parallel

### Import punctuation from the string module

You will use this in your `build_corpus` function to remove punctuation from the corpus.

In [5]:
from string import punctuation

## ⚒️ Implement a pre-processor

Here you will implement a function called `build_corpus` which returns a Corpus object to be used for topic modeling.

The build_corpus function will take a list of texts and return a pre-processed Corpus object. Preprocessing should include the following actions on the corpus using the appropriate functions imported from the corpus module.

 - lemmatize the texts
 - convert tokens to lowercase
 - remove punctuation
 - clean tokens to remove numbers and any tokens shorter than 3 characters

The first part of the function to create the corpus object is done for you. Your job is to call the specific preprocessing functions on the corpus and to return corpus object.


---

### 💡 Note

Loading a corpus as a list of strings is not the only way to use tmtoolkit. Given, for example, a large corpus that might not fit in memory, the current approach would not work well. See the tmtoolkit docs on [working with text corpora](https://tmtoolkit.readthedocs.io/en/latest/text_corpora.html) for more info.

---

In [70]:
def build_corpus(texts, lang="en"):
    """Corpus builder which returns a Corpus object processed on texts as language
    specified by lang (defaults to "en"):

    Should perform all of the following pre-processing functions:

     - Lemmatize the tokens
     - Convert tokens to lowercase
     - Remove punctuation
     - Remove numbers
     - Remove tokens shorter than 2 characters
    """
    # Here, we just use the index of the text as the label for the corpus item
    corpus = Corpus({ i:r for i, r in enumerate(texts) }, language=lang)

    # TODO: Complete the implementation of this function and submit the
    # .py download of this notebook as your assignment submission.
    # lemmatize, to_lowercase, remove_chars, filter_clean_tokens
    lemmatize(corpus)
    to_lowercase(corpus)
    remove_chars(corpus, punctuation)
    # Expected not to find any numeric tokens. Found tokens: ['333']
    filter_clean_tokens(corpus, remove_shorter_than=3, remove_numbers=True)
    return corpus

In [71]:
#~~ /autograde # do not delete this cell

---
### ⚠️  **Caution:** No arbitrary code above this line

The only code written above should be the implementation of your graded function. For experimentation and testing, only add code below.
___

## Function development

Use this section of code to verify your function implementation. You may change the test_corpus as needed to verify your implementation. The grader will be checking that your function returns a Corpus object that meets all of the following critera:

 - tokens are lemmatized
 - tokens are converted to lowercase
 - special characters are removed from tokens
 - tokens shorter than 3 characters and numerics are removed

In [72]:
example_docs = [ # Feel free to edit this corpus for further testing
                # to be sure that your functions meet specifications.
    "The 3 cats sat on the mats!",
    "1 fish 2 fish Red fish Blue fish",
    "She sells $ea$shells"
]
example_corpus = build_corpus(example_docs)
for key in example_corpus:
    print(example_corpus[key])
corpus_tokens_flattened(example_corpus)

Document "0" (3 tokens, 9 token attributes, 2 document attributes)
Document "1" (6 tokens, 9 token attributes, 2 document attributes)
Document "2" (2 tokens, 9 token attributes, 2 document attributes)


['cat',
 'sit',
 'mat',
 'fish',
 'fish',
 'red',
 'fish',
 'blue',
 'fish',
 'sell',
 'eashell']

In [73]:
dtms = {
    "test_corpus": dtm(example_corpus)
}
lda_params = {
    'n_topics': 2,
    'eta': .01,
    'n_iter': 10,
    'random_state': 1234,  # to make results reproducible
    'alpha': 1/16
}

models = compute_models_parallel(dtms, constant_parameters=lda_params)

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
model = models["test_corpus"][0][1]
print_ldamodel_topic_words(model.topic_word_, vocabulary(example_corpus), top_n=5)

topic_1
> #1. fish (0.566384)
> #2. red (0.142655)
> #3. cat (0.142655)
> #4. blue (0.142655)
> #5. sit (0.001412)
topic_2
> #1. sit (0.247549)
> #2. sell (0.247549)
> #3. mat (0.247549)
> #4. eashell (0.247549)
> #5. red (0.002451)


### Assignment submission

After completing the `build_corpus` implementation, download your notebook as a .py file (File > Download > Download .py) and submit the downloaded file for grading.

## Topic modeling Amazon Reviews

Once you have completed the assignment above, you will be well prepared to start your final project for this unit. The project will include loading Amazon reviews into a corpus for topic modeling. The code below demonstrates topic modeling the reviews for a given brand. Note that the final project will require additional segmentation of the data, which is not done for you in the example here.

In [59]:
import gzip
import itertools
import json

asins = []

# To run this code, you will need to download the metadata file from the course
# assets and upload it to your Google Drive. See the notes about that file
# regarding how it was processed from the original file into json-l format.

# META_FILE = "drive/MyDrive/meta_Clothing_Shoes_and_Jewelry.jsonl.gz"
META_FILE = "drive/MyDrive/MSDS_marketing_text_analytics/meta_Clothing_Shoes_and_Jewelry.jsonl.gz"

with gzip.open(META_FILE) as products:
    for product in products:
        data = json.loads(product)
        categories = [c.lower() for c in
                      list(itertools.chain(*data.get("categories", [])))]
        if "nike" in categories:
            asins.append(data["asin"])

### Inspect the first fews ASINs

In [16]:
asins[:3]

['B0000V9K32', 'B0000V9K3W', 'B0000V9K46']

### Check the length, i.e. the number of resulting ASINs

In [60]:
len(asins)

8327

### Compile a selection of review texts

🔥 **Note:** This code as-is uses a reduced version of the reviews file. You may alternatively use the full version of the reviews file but expect it to take a **long time** to process this code.

As with the meta file above, whichever file you use here should be uploaded to the root of your Google Drive.

In [63]:
REDUCED_REVIEWS_FILE = "drive/MyDrive/MSDS_marketing_text_analytics/reviews_Clothing_Shoes_and_Jewelry_5.json.gz"
FULL_REVIEWS_FILE = "drive/MyDrive/MSDS_marketing_text_analytics/reviews_Clothing_Shoes_and_Jewelry.json.gz"

reviews = []
with gzip.open(REDUCED_REVIEWS_FILE) as f:
    for review in f:
        data = json.loads(review)
        if data["asin"] in asins: # This is where we check to see if it is a Nike ASIN
            text = data["reviewText"]
            reviews.append(text)

### Inspect a few of the reviews

In [76]:
for i, review in enumerate(reviews[:5]):
    print(i, review[:80])

0 bought these for supportive shoes after our daughter had ankle surgery. Great fi
1 I was a little hesitant about buying sneakers online, fearing I might end up wit
2 I have a lot of pairs of running shoes, and these are a nice addition to my coll
3 My husband said they are very comfortable and easy to walk in, which he does a l
4 Very nice shoes...my son loved the color and comfort and the lite weight. Would 


### Build a Corpus object from the review texts

In [77]:
reviews_corpus = build_corpus(reviews)

In [78]:
dtms = {
    "reviews_corpus": dtm(reviews_corpus)
}
lda_params = {
    'n_topics': 10,
    'eta': .01,
    'n_iter': 10,
    'random_state': 1234,  # to make results reproducible
    'alpha': 1/16
}

models = compute_models_parallel(dtms, constant_parameters=lda_params)

### Print the topics

In [79]:
model = models["reviews_corpus"][0][1]
print_ldamodel_topic_words(model.topic_word_, vocabulary(reviews_corpus), top_n=5)

topic_1
> #1. size (0.047946)
> #2. fit (0.043153)
> #3. sock (0.038359)
> #4. sneaker (0.029971)
> #5. good (0.025177)
topic_2
> #1. wear (0.042192)
> #2. foot (0.041020)
> #3. good (0.032819)
> #4. nike (0.029304)
> #5. sock (0.026960)
topic_3
> #1. run (0.053057)
> #2. sock (0.051852)
> #3. shoe (0.042207)
> #4. nike (0.034974)
> #5. good (0.030151)
topic_4
> #1. shoe (0.041593)
> #2. great (0.040159)
> #3. buy (0.040159)
> #4. look (0.035858)
> #5. wear (0.027255)
topic_5
> #1. shoe (0.049538)
> #2. nike (0.045728)
> #3. great (0.029220)
> #4. use (0.026680)
> #5. love (0.022870)
topic_6
> #1. shoe (0.063915)
> #2. size (0.043945)
> #3. comfortable (0.037954)
> #4. run (0.033960)
> #5. fit (0.031963)
topic_7
> #1. bag (0.035704)
> #2. love (0.029994)
> #3. feel (0.027139)
> #4. buy (0.020001)
> #5. come (0.017145)
topic_8
> #1. love (0.044609)
> #2. comfortable (0.042322)
> #3. foot (0.033174)
> #4. color (0.028600)
> #5. shoe (0.027456)
topic_9
> #1. shoe (0.043956)
> #2. buy (0.0

## 💾 Save your topic model and review texts for use in Lab 2

Once you have completed the above assignment, run the following code to save your topic model and your review texts to your Google Drive. You will load this model and use it for document classification in Lab 2.

In [82]:
import pickle
from tmtoolkit.topicmod.model_io import save_ldamodel_to_pickle

with open("drive/MyDrive/MSDS_marketing_text_analytics/MSDS_HW2_model.p", "wb") as modelfile:
    save_ldamodel_to_pickle(modelfile, model, vocabulary(reviews_corpus), reviews_corpus.doc_labels, dtm=dtm(reviews_corpus))

In [83]:
with open("drive/MyDrive/MSDS_marketing_text_analytics/MSDS_HW2_corpus.p", "wb") as reviewsfile:
    pickle.dump(reviews, reviewsfile)